In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import distfit as fit
import simpy
from scipy.stats import expon, weibull_min, gamma, lognorm, ttest_ind

Load the dataset that will be used for the simulation

In [ ]:
# Example
df = pd.read_csv('df.csv')

## Data Exploratory

```summary_stat```

This function calculates and returns key summary statistics from the dataset.

Outputs include:
- Number of unique referral IDs
- Outcome distribution for each stage. This will be used to define routing probabilities in the model
- Referral-to-surgery conversion rate


In [ ]:
def summary_stat(df):
    df['Event_Date'] = pd.to_datetime(df['Event_Date'], errors='coerce')

    summary = {}

    # Show count of unique Referral_ID
    summary['Unique Referral IDs'] = df['Referral_ID'].nunique()

    # Distribution of outcomes per stage
    outcome_count = df.groupby('Event_Stage')['Event_Outcome'].value_counts().unstack(fill_value=0)
    
    outcome_percentage = outcome_count.div(outcome_count.sum(axis=1), axis=0).round(2) * 100

    outcome_combine = {}
    for stage, group in df.groupby('Event_Stage'):
        outcome_count = group['Event_Outcome'].value_counts()
        total = outcome_count.sum()

        outcome_combine[int(stage)] = {
            outcome: f"{count} ({(count/total)*100:.2f}%)"
            for outcome, count in outcome_count.items()
        }

    summary['Event Outcome per Stage'] = outcome_combine

    # Conversion rate from "Converted to Referral" to "Knee Procedure"
    converted = df[df['Event_Outcome'] == 'Converted to Referral']['Referral_ID'].unique()
    knee_procedure = df[df['Event_Outcome'].str.contains('Knee Procedure', case=False, na=False)]
    converted_to_knee_procedure = knee_procedure[knee_procedure['Referral_ID'].isin(converted)]['Referral_ID'].nunique()

    if len(converted) > 0:
        convert_rate_str = f"{converted_to_knee_procedure}/{len(converted)} ({(converted_to_knee_procedure/len(converted))*100:.2f}%)"
    else:
        convert_rate_str = "No 'Converted to Referral' cases found"
    
    summary['Referral Conversion to Knee Procedure'] = convert_rate_str             

    return summary

In [ ]:
summary_stat(df)

## Data Preparation

```calculate_stage_duration```

This function calculates the time intervals (in days) between each stage of the patient journey.

Outputs include:
- A summary of stage-to-stage durations, including:
    - Mean
    - Standard deviation
    - Minimum and maximum values
    - Median
    - Count of missing values (indicating patients who finished at certain stages)

The resulting duration_df is used to fit probability distributions for stage durations and serves as the foundation for the simulation model.

In [ ]:
def calculate_stage_duration(df):
    df['Event_Date'] = pd.to_datetime(df['Event_Date'], errors='coerce')

    duration_records = []
    durations = {
        'Stage1_to_Stage2': [],
        'Stage2_to_Stage3': [],
        'Stage3_to_Stage4': [],
        'Stage1_to_Stage4': []
    }
    missing_counts = {
        'Stage1_to_Stage2': 0,
        'Stage2_to_Stage3': 0,
        'Stage3_to_Stage4': 0,
        'Stage1_to_Stage4': 0
    }
    for referral_id, group in df.groupby('Referral_ID'):
        stage_dates = group.set_index('Event_Stage')['Event_Date']

        d1 = stage_dates.get(1)
        d2 = stage_dates.get(2)
        d3 = stage_dates.get(3)
        d4 = stage_dates.get(4)

        record = {'Referral_ID': referral_id,
                  'Stage1': d1,
                  'Stage2': d2,
                  'Stage3': d3,
                  'Stage4': d4}        

        # Stage 1 to Stage 2
        if pd.notna(d1) and pd.notna(d2):
            diff = (d2-d1).days
            record['Stage1_to_Stage2'] = diff
            durations['Stage1_to_Stage2'].append(diff)
        else:
            record['Stage1_to_Stage2'] = None
            missing_counts['Stage1_to_Stage2'] += 1

        # Stage 2 to Stage 3
        if pd.notna(d2) and pd.notna(d3):
            diff = (d3-d2).days
            record['Stage2_to_Stage3'] = diff
            durations['Stage2_to_Stage3'].append(diff)
        else:
            record['Stage2_to_Stage3'] = None
            missing_counts['Stage2_to_Stage3'] += 1

        # Stage 3 to Stage 4
        if pd.notna(d3) and pd.notna(d4):
            diff = (d4-d3).days
            record['Stage3_to_Stage4'] = diff
            durations['Stage3_to_Stage4'].append(diff)
        else:
            record['Stage3_to_Stage4'] = None
            missing_counts['Stage3_to_Stage4'] += 1

    duration_df = pd.DataFrame(duration_records)

    summary = {}
    output_str = ""
    for key, values, in durations.items():
        if values:
            stats = pd.Series(values).agg(['mean', 'std', 'min', 'max', 'median']).round(2).to_dict()
        else:
            stats = {'mean': None, 'std': None, 'min': None, 'max': None}
        stats['missing_dates_skipped'] = missing_counts[key]
        summary[key] = stats

        output_str += f"---{key}---\n"
        for stat_key, value in stats.items():
            output_str += f"{stat_key}: {value}\n"
        output_str += "\n"
    print(output_str)

    return summary, duration_df                     

In [ ]:
duration_summary, duration_df = calculate_stage_duration(df) 

```arrival_rate```

In the simulation, the arrival rate need to be be determined to model the inflow of referrals.

The arrival rate (patients per day) is calculated by dividing the total number of unique referral IDs by the total observation period:
Arrival Rate =  1678/365 ≈ 4.6

The arrival rate (per day) would be λ (in Poisson distribution).

In [ ]:
unique_referrals = df['Referral_ID'].nunique()
observed_days = 365 # Set to 1 year, as the dataset represents referral arrivals over a one-year period
arrival_rate = unique_referrals/observed_days

print(f"Total Referrals: {unique_referrals}")
print(f"Total Days Observed: {observed_days}")
print(f"Arrival Rate (per day): {arrival_rate}")

```goodness_of_fit_test```

To identify the most suitable probability distribution that reflects the process, a goodness-of-fit test was performed using the ```distfit``` package.

In [ ]:
def goodness_of_fit_test(df, duration_cols, plot=True):
    fitted_params = {}

    for col in duration_cols:
        data = df[col].dropna().values

        dfit = fit.distfit(distr=["lognorm", "weibull_min", "gamma", "expon"], alpha=0.05, smooth=5, bound='both')
        # For the distribution fitting, additional distributions can be explored by adding their names to the list
        dfit.fit_transform(data)

        best_fit = dfit.model
        dist_name = best_fit["name"]
        params = best_fit["params"]
       
        if len(params) == 3:
            shape, loc, scale = params
        elif len(params) == 2:
            loc, scale = params
            shape = None
        else:
            shape = loc = scale = None

        # Save parameters for later use
        fitted_params[col] = {
            "distribution": dist_name,
            "loc": loc,
            "scale": scale,
            "shape": shape
        }

        # Plot
        if plot:
            fig, axs = plt.subplots(2,2, figsize=(18,12))
            dfit.plot(chart='pdf', n_top=5, ax=axs[0,0])
            axs[0,0].set_title(f'PDF - {col}')
            
            dfit.plot(chart='cdf', n_top=5, ax=axs[0,1])
            axs[0,1].set_title(f'CDF - {col}')

            dfit.qqplot(data, n_top=5, ax=axs[1,0])
            axs[1,0].set_title(f'QQ Plot - {col}')

            axs[1,1].axis('off')
            plt.tight_layout()
            plt.show()
    return fitted_params 
     

In [ ]:
duration_cols = ['Stage1_to_Stage2', 'Stage2_to_Stage3', 'Stage3_to_Stage4']
fitted_results = goodness_of_fit_test(duration_df, duration_cols, plot=True)  

## Simulation

```get_duration```

This function selects the most suitable probability distribution based on the results of the goodness-of-fit test.

In [10]:
def get_duration(stage_name):
    dist_info = fitted_results[stage_name]
    dist = dist_info['distribution']
    loc = dist_info['loc']
    scale = dist_info['scale']
    shape = dist_info['shape']

    adjusted_loc = scale + loc
    while True:
        if dist == 'expon':
            val = expon.rvs(loc=loc, scale=scale)
        elif dist == 'norm':
            val = norm.rvs(loc=loc, scale=scale)
        elif dist == 'gamma':
            val = gamma.rvs(shape, loc=loc, scale=scale)
        elif dist == 'weibull_min':
            val = weibull_min.rvs(shape, loc=loc, scale=scale)
        elif dist == 'triang':
            val = triang.rvs(shape, loc=loc, scale=scale)
        elif dist == 'lognorm':
            val = lognorm.rvs(shape, loc=loc, scale=scale)
        else:
            raise ValueError(f"Unsupported distribution: {dist}")
        
        if val >= 0:
            return val

```patient_arrivals``` 

This function simulates patient arrivals using an exponential distribution with the previously calculated arrival rate (λ).

In [11]:
def patient_arrivals(env, arrival_rate, resources, arrival_duration=365):
    i = 0
    while True:
        if env.now > arrival_duration:
            break
        interarrival_time = np.random.exponential(1 / arrival_rate) ## Using exponential distribution for the patient arrivals
        
        if env.now + interarrival_time> arrival_duration:
            break
        yield env.timeout(interarrival_time)
        env.process(patient_process(env, f"Patient {i+1}", resources))
        i += 1

```queue_lengths``` 

This function tracks and records the queue lengths at different stages during the simulation run.

In [12]:
def queue_length(env, resources, name, stage):
    for stage_name, resource in resources.items():
        queue_len = len(resource.queue)
        busy = resource.count
        queue_records.append({
            'Time': env.now,
            'Name': name,
            'Stage': stage,
            'Resource': stage_name,
            'Queue_Length': queue_len,
            'Busy': busy
        })

```patient_process``` 

This function represents the conceptual patient pathway, modeling the flow of patients through each stage of the system.

In [ ]:
def patient_process(env, name, resources):
    start = env.now
    record = {'Name': name, 'Arrival_Time': start}
    print(f"{name} arrives at {env.now:.2f}")

    # Stage 1 → Stage 2
    with resources['gp'].request() as req:
        queue_length(env, resources, name, 'Stage1_to_Stage2_wait_start')
        yield req
        queue_length(env, resources, name, 'Stage1_to_Stage2_wait_end')
        yield env.timeout(get_duration('Stage1_to_Stage2'))
    record['Stage2'] = env.now
    record['Stage1_to_Stage2'] = record['Stage2'] - record['Arrival_Time']
    queue_length(env, resources, name, 'Stage1_to_Stage2')
    print(f"{name} completed Stage 1 to 2 at {env.now:.2f}")

    # Stage 2 routing
    # Define possible outcomes for Stage 2 of the patient pathway
    stage2_choices = ["Converted to Referral", "Advice from GP", "Unknown"] # Add new outcomes here if needed
    stage2_probs = [0.9082, 0.0745, 0.0173] # Corresponding probabilities for each outcome (must sum to 1)
    outcome2 = np.random.choice(stage2_choices, p=stage2_probs)
    record['Outcome_Stage2'] = outcome2

    if outcome2 != "Converted to Referral":
        record['Dropped_At'] = 'Stage2'
        record['End'] = env.now
        patient_results.append(record)
        print(f"{name} dropped at Stage 2 ({outcome2})")
        return

    # Stage 2 → Stage 3
    with resources['outpatient'].request() as req:
        queue_length(env, resources, name, 'Stage2_to_Stage3_wait_start')
        yield req
        queue_length(env, resources, name, 'Stage2_to_Stage3_wait_end')
        yield env.timeout(get_duration('Stage2_to_Stage3'))
    record['Stage3'] = env.now
    record['Stage2_to_Stage3'] = record['Stage3'] - record['Stage2']
    queue_length(env, resources, name, 'Stage2_to_Stage3')
    print(f"{name} completed Stage 2 to 3 at {env.now:.2f}")

    # Stage 3 routing
    # Define possible outcomes for Stage 3 of the patient pathway
    stage3_choices = [
        "Follow-Up but no recorded treatment",
        "Discharged - 1st Outpatient",
        "No Appointment Found",
        "Follow-Up and recorded treatment"]  # Add new outcomes here if needed
    stage3_probs = [0.4370, 0.2822, 0.1430, 0.1378] # Corresponding probabilities for each outcome (must sum to 1)
    outcome3 = np.random.choice(stage3_choices, p=stage3_probs)
    record['Outcome_Stage3'] = outcome3

    if outcome3 != "Follow-Up and recorded treatment":
        record['Dropped_At'] = 'Stage3'
        record['End'] = env.now
        patient_results.append(record)
        print(f"{name} dropped at Stage 3 ({outcome3})")
        return

    # Stage 3 → Stage 4
    with resources['knee'].request() as req:
        queue_length(env, resources, name, 'Stage3_to_Stage4_wait_start')
        yield req
        queue_length(env, resources, name, 'Stage3_to_Stage4_wait_end')
        yield env.timeout(get_duration('Stage3_to_Stage4'))
    record['Stage4'] = env.now
    record['Stage3_to_Stage4'] = record['Stage4'] - record['Stage3']
    record['Stage1_to_Stage4'] = record['Stage4'] - record['Arrival_Time']
    queue_length(env, resources, name, 'Stage3_to_Stage4')
    record['Outcome_Stage4'] = 'Knee Procedure'
    print(f"{name} completed Stage 3 to 4 (Knee Procedure) at {env.now:.2f}")

    record['End'] = env.now
    patient_results.append(record)


```run_simulation```

This function executes the entire simulation, coordinating patient arrivals, stage processes, and tracking key metrics such as queue lengths and waiting times.

In [ ]:
def run_simulation(simulation_time, arrival_rate):
    global patient_results, queue_records
    patient_results = []
    queue_records = []

    # Initialize the simulation environment
    env = simpy.Environment()

    # Define resources with adjusted capacities
    resources = {
        'gp': simpy.Resource(env, capacity=1000), 
        'outpatient': simpy.Resource(env, capacity=1000),
        'knee': simpy.Resource(env, capacity=100)
    }
    
    # The duration between stages includes waiting time + service time until the outcome.
    # Need to adjust the resource capacity by calculating as:
        # Number of resources x Working days x Patients each resource can handle simultaneously

    # GP & Outpatient stages:
        # - 10 resources, 20 working days, 5 patients per resource at a time
        # - Capacity: 10 x 20 x 5 = 1000
        # - Assumption: staff can handle multiple patients at once due to back-and-forth flow

    # Knee Surgeon stage:
        # - resource, 20 working days, 5 patients per resource
        # - Capacity: 1 x 20 x 5 = 100
        # - Assumption: staff can handle multiple patients at once due to back-and-forth flow

    
    # Start the patient arrivals process
    env.process(patient_arrivals(env, arrival_rate, resources))

    # Run the simulation
    env.run()

    return pd.DataFrame(patient_results), pd.DataFrame(queue_records)

Run the simulation

In [ ]:
simulation_time = 365 # The period of how long the simulation will be
warmup_time = 0
total_simulation_time = warmup_time + simulation_time

all_runs = []
all_queue_runs = []
no_of_runs = 50 # No of simulation runs

for i in range(no_of_runs):
    sim_result, queue_result = run_simulation(simulation_time=total_simulation_time, arrival_rate=arrival_rate)
    sim_result['Run'] = i + 1  
    queue_result['Run'] = i + 1
    all_runs.append(sim_result)
    all_queue_runs.append(queue_result)

# Combine all runs into a single DataFrame
all_runs_df = pd.concat(all_runs, ignore_index=True)
all_queue_runs_df = pd.concat(all_queue_runs, ignore_index=True)

```t_test```

This function compares the real-world data with the simulated data using a t-test to determine if there is a statistically significant difference between them.

In [16]:
def t_test(df, simulation_df, column, plot=True):

    # Drop NA values if any
    emp_data = df[column].dropna().values
    sim_data = simulation_df[column].dropna().values

    # Perform t-test
    t_stat, p_value = ttest_ind(emp_data, sim_data, equal_var=False)

    print(f"T-Test for '{column}'")
    print(f"Statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print("→ Significant difference between empirical and simulated data.")
    else:
        print("→ No significant difference (simulation is a good match).")

    # Plot
    if plot:
        plt.figure(figsize=(8, 5))
        sns.kdeplot(emp_data, label='Empirical', fill=True, linewidth=2)
        sns.kdeplot(sim_data, label='Simulated', fill=True, linewidth=2)
        plt.title(f"KDE Plot: Empirical vs Simulated ({column})")
        plt.xlabel(column)
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    return t_stat, p_value

In [ ]:
t_test(duration_df, all_runs_df, column='Stage1_to_Stage2', plot=True)
t_test(duration_df, all_runs_df, column='Stage2_to_Stage3', plot=True)
t_test(duration_df, all_runs_df, column='Stage3_to_Stage4', plot=True)

## Summary of Simulated Data

Get the summary of the simulated stage-to-stage durations, including:

 - Mean
 - Standard deviation
 - Median
 - Minimum and maximum values
 - Lower & upper CI

In [ ]:
cols = ['Stage1_to_Stage2', 'Stage2_to_Stage3', 'Stage3_to_Stage4']

all_runs_summary = all_runs_df[cols].agg(['mean', 'std', 'median', 'min', 'max']).T.round(2)

n = all_runs_df[cols].count()

# t critical value for 95% CI, two-tailed
confidence = 0.95
alpha = 1 - confidence
t_crit = stats.t.ppf(1 - alpha/2, df=n - 1)

# Calculate margin of error
margin_of_error = t_crit * all_runs_summary['std'] / np.sqrt(n)

# Add lower and upper bound columns for the CI
all_runs_summary['CI_lower'] = (all_runs_summary['mean'] - margin_of_error).round(2)
all_runs_summary['CI_upper'] = (all_runs_summary['mean'] + margin_of_error).round(2)

# Round other stats as well
all_runs_summary = all_runs_summary.round(2)

print(all_runs_summary)

Get the simulated length of queue of each stage-to-stage

In [ ]:
stage_stats = (
    all_queue_runs_df.groupby(['Run', 'Stage'])
    .agg(
        avg_queue=('Queue_Length', 'mean'),
        max_queue=('Queue_Length', 'max'),
        avg_busy=('Busy', 'mean')
    )
    .reset_index()
)

queue_max = stage_stats.groupby("Stage")["max_queue"].max().reset_index()
queue_mean_max = stage_stats.groupby("Stage")["max_queue"].mean().reset_index()
print(queue_max)
print(queue_mean_max)